In [1]:
%load_ext autoreload
%autoreload 2

In [8]:
from isce2_topsapp import download_slcs, download_orbits, download_dem_for_isce2, download_aux_cal
from isce2_topsapp.__main__ import check_esa_credentials

2023-11-16 18:00:22,356 - matplotlib.font_manager - DEBUG - Using fontManager instance from /Users/cmarshak/.matplotlib/fontlist-v330.json


# Input Data

In [3]:
data = {
        "reference_scenes": ["S1B_IW_SLC__1SDV_20210723T014947_20210723T015014_027915_0354B4_B3A9"],
        "secondary_scenes": ["S1B_IW_SLC__1SDV_20210711T014922_20210711T014949_027740_034F80_859D",
                             "S1B_IW_SLC__1SDV_20210711T014947_20210711T015013_027740_034F80_D404",
                             "S1B_IW_SLC__1SDV_20210711T015011_20210711T015038_027740_034F80_376C"]
        }
reference_scenes = data['reference_scenes']
secondary_scenes = data['secondary_scenes']

In [4]:
dry_run = True

# SLC

Do everything but download SLCs
*Note, if you encounter `NetrcParseError: access too permissive` error, try to update permissions with `chmod 0600 ~/.netrc`

In [5]:
out_slc = download_slcs(reference_scenes,
                        secondary_scenes,
                        dry_run=dry_run)
out_slc

2023-11-16 17:35:54,152 - fiona.env - DEBUG - PROJ data files are available at built-in paths.
2023-11-16 17:35:54,172 - fiona.env - DEBUG - Entering env context: <fiona.env.Env object at 0x19ab7e850>
2023-11-16 17:35:54,173 - fiona.env - DEBUG - Starting outermost env
2023-11-16 17:35:54,173 - fiona.env - DEBUG - No GDAL environment exists
2023-11-16 17:35:54,174 - fiona.env - DEBUG - New GDAL environment <fiona._env.GDALEnv object at 0x19a179be0> created
2023-11-16 17:35:54,198 - fiona._env - DEBUG - GDAL_DATA found in environment.
2023-11-16 17:35:54,199 - fiona._env - DEBUG - PROJ data files are available at built-in paths.
2023-11-16 17:35:54,199 - fiona._env - DEBUG - Started GDALEnv: self=<fiona._env.GDALEnv object at 0x19a179be0>.
2023-11-16 17:35:54,200 - fiona.env - DEBUG - Updated existing <fiona._env.GDALEnv object at 0x19a179be0> with options {}
2023-11-16 17:35:54,200 - fiona.env - DEBUG - Entered env context: <fiona.env.Env object at 0x19ab7e850>
2023-11-16 17:35:54,203 

{'ref_paths': ['S1B_IW_SLC__1SDV_20210723T014947_20210723T015014_027915_0354B4_B3A9.zip'],
 'sec_paths': ['S1B_IW_SLC__1SDV_20210711T015011_20210711T015038_027740_034F80_376C.zip',
  'S1B_IW_SLC__1SDV_20210711T014947_20210711T015013_027740_034F80_D404.zip',
  'S1B_IW_SLC__1SDV_20210711T014922_20210711T014949_027740_034F80_859D.zip'],
 'extent': [-119.080994, 33.405682, -115.988029, 35.43090952856198],
 'gunw_geo': <POLYGON ((-118.731 33.534, -119.051 34.9, -119.051 34.9, -119.052 34.903, -...>,
 'processing_extent': [-119.080994, 33.405682, -115.988029, 35.43090952856198],
 'reference_properties': [{'beamModeType': 'IW',
   'browse': None,
   'bytes': 4334545363,
   'centerLat': 34.4266,
   'centerLon': -117.5237,
   'faradayRotation': None,
   'fileID': 'S1B_IW_SLC__1SDV_20210723T014947_20210723T015014_027915_0354B4_B3A9-SLC',
   'flightDirection': 'ASCENDING',
   'groupID': 'S1B_IWDV_0107_0114_027915_064',
   'granuleType': 'SENTINEL_1B_FRAME',
   'insarStackId': None,
   'md5sum': '

# Orbits
Make sure `ESA_USERNAME` and `ESA_PASSWORD` environment variables are set before running this.

In [10]:
check_esa_credentials(None, None)
out_orbit = download_orbits(reference_scenes,
                            secondary_scenes,
                            dry_run=dry_run)
out_orbit

{'orbit_directory': 'orbits',
 'reference_orbits': ["https://scihub.copernicus.eu/gnss/odata/v1/Products('b1085082-4941-4d7e-8e15-4ab9fa6bd118')/$value"],
 'secondary_orbits': ["https://scihub.copernicus.eu/gnss/odata/v1/Products('0a5b7c24-01de-4c15-a313-2ff02102f995')/$value"]}

# DEM

Don't currently have "dry-run" for this localization.

In [ ]:
out_dem = download_dem_for_isce2(out_slc['extent'])
out_dem

Reading glo_30 Datasets: 100%|█| 15/15 [00:20<00:00,


# Aux Cal

In [ ]:
out_aux_cal = download_aux_cal()

# Combining Metadata

In [ ]:
out = {**out_slc,
       **out_dem,
       **out_aux_cal,
       **out_orbit}
out

# Check Name

In [ ]:
from isce2_topsapp.packaging import get_gunw_id

gunw_id = get_gunw_id(reference_properties=out['reference_properties'],
                      secondary_properties=out['secondary_properties'],
                      extent=out['extent'])
gunw_id
